# Blood Smear Classification

We'll compare the performance of a dense neural network and of a convolutional neural network with a dense top for the classification of blood smears in a dataset of blood smears containing blood cells with and without malaria. 

## Load Malaria Dataset

We load and uncompress the malaria dataset available at https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip.

This dataset was origianlly published in S. Rajaraman, S. K. Antani, M. Poost- chi, K. Silamut, Md A. Hossain, R. J. Maude, S. Jaeger, and G. R. Thoma. _Pre-trained convolutional neural networks as feature extractors toward improved malaria parasite detection in thin blood smear images._ PeerJ 6, e4568, 2018.

In [ ]:
import os
from torchvision.datasets.utils import download_url, _extract_zip

dataset_path = os.path.join(".", "blood_smears_dataset")
if not os.path.exists(dataset_path):
    url = "https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip"
    download_url(url, ".")
    _extract_zip("cell_images.zip", dataset_path, None)
    os.remove("cell_images.zip")

We now obtain the paths to the files containing the `Infected` and `Parasitized` images.

In [ ]:
import glob

base_dir = os.path.join(dataset_path, "cell_images")
uninfected_files = glob.glob(os.path.join(base_dir, "Uninfected", "*.png"))
parasitized_files = glob.glob(os.path.join(base_dir, "Parasitized", "*.png"))


## Visualize Data

We impleemnt the function `plot_blood_smears`, save it in `fnc_blood_smears.py` ...

```python
def plot_blood_smears(title, files):
    import matplotlib.pyplot as plt

    fig, axs = plt.subplots(3, 3, figsize=(8, 8))
    for i, ax in enumerate(axs.ravel()):
        image = plt.imread(files[i])
        ax.imshow(image)
        
    fig.suptitle(title, fontsize=16)
    plt.show()
```

... and use it to visualize some of the data.

In [ ]:
from fnc_blood_smears import plot_blood_smears

plot_blood_smears("Uninfected", uninfected_files)
plot_blood_smears("Parasitized", parasitized_files)

## Preprocess Data

We define a transformation to resize the images to 28 by 28 pixels and convert them to PyTorch tensors (note that `ToTensor()` also normalizes their values between 0 and 1).

In [ ]:
from torchvision.transforms import Compose, Resize, ToTensor

input_transform = Compose([Resize((28, 28)), ToTensor()])

We then define a transformation to make the target equal to 0 for the uninfected cells and to 1 for the parasitized ones, achieved through `abs(1 - target)`. This step is followed by converting it into a PyTorch tensor using `tensor()`, ensuring the data is in the correct format for PyTorch operations. Next, we convert the tensor into a floating-point type with `float()`, enhancing its compatibility with PyTorch's computational requirements. Finally, we add a new dimension to the tensor using `unsqueeze(-1)`, preparing it for batch processing in neural network models.

In [ ]:
from torch import tensor

def target_transform(target):
    return tensor(abs(1 - target)).float().unsqueeze(-1)

We then set up a dataset where images are loaded from a structured directory (`base_dir`).

In [ ]:
from torchvision.datasets import ImageFolder

dataset = ImageFolder(base_dir, 
                      transform=input_transform, 
                      target_transform=target_transform)

We extract a subset with 5,000 of the ca. 27,000 cell images for computational efficiency, and split it into `train` and `test` sets.

In [ ]:
from torch import randperm
from torch.utils.data import random_split, Subset

images_num = 5000
images_idx = randperm(len(dataset))[:images_num]
subset = Subset(dataset, images_idx)

train_size = int(0.8 * len(subset))
test_size = len(subset) - train_size
train, test = random_split(subset, [train_size, test_size])

Finally, we define the dataloaders for both sets. For the training, we'll set `batch_size=32`, while we'll set a larger batch for the testing.

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train, batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=124, shuffle=False)

## Dense Neural Network

We define a dense neural network (DNN) using `deeplay` with 2 layers with 128 neurons.

In [ ]:
import deeplay as dl
from torch.nn import Sigmoid

dnn = dl.MultiLayerPerceptron(in_features=28 * 28 * 3,
                               hidden_features=[128, 128],
                               out_features=1,
                               out_activation=Sigmoid)

print(dnn)

We then define a classifier using the DNN ...

In [ ]:
import torchmetrics as tm

dnn_classifier_template = dl.BinaryClassifier(model=dnn,
                                              optimizer=dl.RMSprop(lr=.001))

dnn_classifier = dnn_classifier_template.create()

print(dnn_classifier)

... and a trainer.

In [ ]:
dnn_trainer = dl.Trainer(max_epochs=20, accelerator="auto")

### Train

We train the DNN classifier ...

In [ ]:
dnn_trainer.fit(dnn_classifier, train_loader)

... and evaluate its performance over the training set.

In [ ]:
_ = dnn_trainer.test(dnn_classifier, test_loader)

### Plot ROC Curve

We implement the `plot_roc`, save it in `fnc_blood_smears.py` ...

```python
def plot_roc(classifier, dataset):
    from torch import tensor, stack
    from sklearn.metrics import roc_curve, auc
    from matplotlib import pyplot as plt

    # calculate predictions
    images, gt = zip(*dataset)
    pred = classifier(tensor(stack(images))).tolist()
    
    # calculate the ROC curve
    fpr, tpr, thresholds = roc_curve(gt, pred, pos_label=1) 
    roc = auc(fpr, tpr) 

    # plot the ROC curve
    plt.figure(figsize=(5, 5))
    plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc:.3f})", linewidth=2)
    plt.title("ROC curve")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.axis("square")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.legend(loc = 'center right')
    plt.show()

    return images, gt, pred, roc
```

... and use it to visualize the receiver operator curve (ROC). Furthermore, this function returns the test images, the groundtruths, the predictions, and the ROC.

In [ ]:
from fnc_blood_smears import plot_roc

_ = plot_roc(classifier=dnn_classifier, dataset=test)

## Convolutional Neural Network with a Dense Top

We'll now build a convolutional neural network a dense top using `deeplay` ...


In [ ]:
from torch.nn import AdaptiveAvgPool2d, MaxPool2d

cnn = dl.Sequential(
    dl.ConvolutionalNeuralNetwork(
        in_channels=3, 
        hidden_channels=[32, 32, 64], 
        out_channels=64,
    ),
    dl.Layer(AdaptiveAvgPool2d, output_size=1),
    dl.MultiLayerPerceptron(
        in_features=64, 
        hidden_features=[], 
        out_features=1,
        out_activation=Sigmoid,
    )
)
cnn[0].blocks[2].pool.configure(MaxPool2d, kernel_size=2)

... define a classifier using the CNN ...

In [ ]:
cnn_classifier_template = dl.BinaryClassifier(
    model=cnn, 
    optimizer=dl.RMSprop(lr=.001),
)

cnn_classifier = cnn_classifier_template.create()

print(cnn_classifier)

... train it ...

In [ ]:
cnn_trainer = dl.Trainer(max_epochs=20, accelerator="auto")

cnn_trainer.fit(cnn_classifier, train_loader)

... evaluate the performance over the test set ...

In [ ]:
_ = cnn_trainer.test(cnn_classifier, test_loader)

... and display the ROC curve with the AUC value.

In [ ]:
images, gt, pred, _ = plot_AUROC(classifier=cnn_classifier, dataset=test)

## Failure analysis

We'll use a function to visualize some of the wrongly classified cells, looking for common patterns.

```python

```

In [ ]:
def plot_failures(images, gt, pred, threshold=.5, plot_num=5):
    from matplotlib import pyplot as plt 
    from numpy import array, squeeze
    
    pred = array(pred).squeeze()
    gt = array(gt).squeeze()
    images = array(images)

    false_positives = (pred > threshold) & (gt == 0)
    false_positives_images = images[false_positives]

    false_negatives = (pred < threshold) & (gt == 1)
    false_negatives_images = images[false_negatives]

    plt.figure(figsize=(plot_num * 2, 5))
    for i in range(plot_num):
        # false positives
        plt.subplot(2, plot_num, i + 1)
        plt.imshow(false_positives_images[i].transpose(1, 2, 0))
        if i == 0:
            plt.title("False positives", fontsize=16, y=1.1)

        # false negatives
        plt.subplot(2, plot_num, plot_num + i + 1)
        plt.imshow(false_negatives_images[i].transpose(1, 2, 0))
        if i == 0:
            plt.title("False negatives", fontsize=16, y=1.1)

    plt.tight_layout()
    plt.show()

In [ ]:
#from fnc_blood_smears import plot_failure

plot_failures(images, gt, pred)

## Access Filters

We can access the filters used by the neural network. The follwoing code accesses the 32nd filter's weights in the first convolutional layer of the CNN. It navigates through the model's first module (`model[0]`), selects the initial block (`blocks[0]`), and then targets the layer's weights (`layer.weight[31]`). 

In [ ]:
filter = cnn_classifier.model[0].blocks[0].layer.weight[31]

print(filter)

## Activations and Grad-CAM

To visualize the network features, we'll use `hooks`, functions that allows us to access the information that the model sees during forward and backward passes, such as activations and gradients, respectively. We'll define them as context manager classes, so that we can use them with the `with` statement:

```python

```

```python

```

The use of context managers (`with` statements) for hooks is a good practice. It ensures that the hooks are properly set up and removed when they are no longer needed, preventing potential side effects or memory leaks. This is particularly important in complex neural network models where maintaining computational efficiency and correctness is crucial.

We'll randomly pick the image of an infected smear.

In [ ]:
import numpy as np
from torch import cat

# Convert the ground truth labels (gt) to a categorical format and then to a list.
gt_as_list = (cat(gt) == 1).tolist()

# Find the indices where the ground truth labels are equal to 1.
indices = np.where(gt_as_list)[0]

# Randomly select one index from those indices.
index = np.random.choice(indices, 1)[0]

# Retrieve the image corresponding to the selected index.
image = images[index]

Activations and gradients at a specific layer can be obtained from the forward and backward pass, respectively ...

In [ ]:
class fwd_hook():
    # Initializer method for the fwd_hook class.
    def __init__(self, model):
        # Register a forward hook to the provided model. The hook is set to the hook_func method.
        # This hook will be called every time the forward method of the model is executed.
        self.hook = model.register_forward_hook(self.hook_func)

    # The forward hook function.
    def hook_func(self, model, i, o):
        # This function is called during the forward pass of the model.
        # 'model' is the model to which the hook is attached.
        # 'i' is the input to the model's layer.
        # 'o' is the output from the model's layer.

        # Print a message indicating the forward hook is running.
        print("Forward hook running ...") 

        # Detach the output 'o' from the current computation graph and clone it.
        # This prevents any modifications to 'o' from affecting the original tensor,
        # and avoids potential issues with backpropagation.
        self.stored = o.detach().clone()

        # Print the size of the activations (the output tensor).
        print(f"Activations size: {self.stored.size()}")

    # Enter method for the context manager.
    def __enter__(self, *args): 
        # When entering the context (using the 'with' statement), return this object.
        return self
    
    # Exit method for the context manager.
    def __exit__(self, *args): 
        # When exiting the context, remove the forward hook from the model.
        # This is important for resource management and to ensure that the hook
        # does not remain active beyond its intended scope.
        self.hook.remove()

In [ ]:
class bwd_hook():
    # Initializer method for the bwd_hook class.
    def __init__(self, model):
        # Register a full backward hook to the provided model. The hook is set to the hook_func method.
        # This hook will be called every time the backward pass of the model is executed.
        self.hook = model.register_full_backward_hook(self.hook_func)

    # The backward hook function.
    def hook_func(self, model, gi, go):
        # This function is called during the backward pass of the model.
        # 'model' is the model to which the hook is attached.
        # 'gi' represents the gradients with respect to the input.
        # 'go' represents the gradients with respect to the output.

        # Print a message indicating the backward hook is running.
        print("Backward hook running ...")

        # Detach the first output gradient 'go[0]' from the current computation graph and clone it.
        # This prevents any modifications to 'go[0]' from affecting the original tensor,
        # and avoids potential issues with backpropagation.
        # Note: go[0] is used because go is a tuple of gradients, one for each output.
        self.stored = go[0].detach().clone()

        # Print the size of the stored gradients.
        print(f"Gradients size: {self.stored.size()}")

    # Enter method for the context manager.
    def __enter__(self, *args): 
        # When entering the context (using the 'with' statement), return this object.
        # This enables the use of the 'with' statement with instances of this class.
        return self
    
    # Exit method for the context manager.
    def __exit__(self, *args): 
        # When exiting the context, remove the backward hook from the model.
        # This is important to ensure that the hook is only active during the intended scope
        # and to release any resources associated with the hook.
        self.hook.remove()

In [ ]:
#from fnc_blood_smears import fwd_hook, bwd_hook

layer = cnn_classifier.model[0].blocks[3].layer

# Using backward and forward hooks on the specified layer
with bwd_hook(layer) as bh, fwd_hook(layer) as fh:
    # Forward pass through the model with the given image
    pred = cnn_classifier.model(image.unsqueeze(0))
    # Executing the backward pass for backpropagation
    pred.backward()

activations = fh.stored
gradients = bh.stored

... we can plot the activations ...

In [ ]:
def plot_activations(activations, cols=8):
    from matplotlib import pyplot as plt

    rows = -(activations.shape[0] // -cols)

    fig, axs = plt.subplots(rows, cols, figsize=(2 * cols, 2 * rows))
    for i, ax in enumerate(axs.ravel()):
        ax.axis('off')
        if i < activations.shape[0]:
            ax.imshow(activations[i].numpy())
            ax.set_title(i)

    fig.tight_layout()
    plt.show()

In [ ]:
#from fnc_blood_smears import plot_activations

plot_activations(activations[0])

... or combine gradients and activations to calculate Grad-CAM and inspect on which part of an image the CNN focuses on to predict its outputs. 

In [ ]:
import matplotlib.image as mpimg

image_hr = mpimg.imread(dataset.imgs[subset.indices[test.indices[index]]][0])

In [ ]:
def plot_gradcam(image, grad_cam):
    from matplotlib import pyplot as plt
    import skimage

    grad_cam = skimage.transform.resize(grad_cam, image.shape, order=2)
    grad_cam = skimage.exposure.rescale_intensity(grad_cam, out_range=(0.25, 1))

    plt.figure(figsize=(12, 5)) 

    plt.subplot(1, 3, 1)
    plt.imshow(image, interpolation="bilinear")
    plt.title("Original image")
    plt.axis("off")
    
    plt.subplot(1, 3, 2)
    plt.imshow(grad_cam.mean(axis=-1), interpolation="bilinear")
    plt.title("Grad-CAM")
    plt.axis("off")
    
    plt.subplot(1, 3, 3)
    plt.imshow(image * grad_cam)
    plt.title("Overlay")
    plt.axis("off")
    
    plt.tight_layout()
    plt.show()

In [ ]:
from torch.nn.functional import relu

pooled_gradients = gradients[0].mean(dim=[1,2], keepdim=True)
grad_cam = relu((pooled_gradients * activations[0]).sum(0)).detach().numpy()

#from fnc_blood_smears import plot_gradcam

plot_gradcam(image_hr, grad_cam)